# Objective: In this model, the additional dataset has been used to create new features and the model is trained using these features. We are using one coupling type to compare four models-LightGBM, Random Forest, XGBoost, CatBoost.

## 1a. Loading Data and Packages  

In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /home/saritacharde04/anaconda3/lib/python3.6/site-packages (19.2.3)


In [2]:
from category_encoders import OrdinalEncoder, OneHotEncoder
import eli5
#from eli5.sklearn import PermutationImportance

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import randint, uniform

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial
from tqdm import tqdm_notebook as tqdm


In [3]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [4]:
trainval = pd.read_csv('../datasets/train/train.csv')
test = pd.read_csv('../datasets/test/test.csv')
structures = pd.read_csv('../datasets/train/structures.csv')
dipole = pd.read_csv('../datasets/train/dipole_moments.csv')
contrib = pd.read_csv('../datasets/train/scalar_coupling_contributions.csv')
magnetic = pd.read_csv('../datasets/train/magnetic_shielding_tensors.csv')
mulliken = pd.read_csv('../datasets/train/mulliken_charges.csv')
potential_energy = pd.read_csv('../datasets/train/potential_energy.csv')


# 1b. Inference from studying the data and exploratory data analysis(EDA)
a. Looking at the data, we can see that the train and test sets had relatively even distributions of scalar coupling type and of the number of atoms present in each dataset. This tells us that the train data is a good enough representation of the test data in order to create a model that predicts the scalar coupling constants. b. The distribution of the scalar coupling constant values isolated by type also reveals that there are clear differences in the ranges that these values appear in. This gives us the insight that different molecular properties affect each type of J coupling differently and unique models should be used for all 8 coupling types found in the dataset. c. The test set structure is the same as the training except that we don't have the scalar_coupling_constant column. Among the other datasets, the most promising seems to be structures.csv, as it is the only one that is available for both training and testing sets. All the other dataframes are available only for the former. d. It is mentioned in the problem statement that the molecules listed for train and test are totally different. e. there are no null values for both train and test

## 2. Reduce Memory Function

In [5]:
def reduce_mem_usage(df, verbose=True):
    """
    This function reduces the numeric to the least possible numeric type that fits the data so 
    memory usage during transforming and training will be reduced.
    Taken from: https://www.kaggle.com/todnewman/keras-neural-net-for-champs
    
    Han
    Parameters:
    ===========
    dataframe: input dataframe 
    verbose: verbose mode, default True.
    Output:
    ===========
    dataframe: dataframe with numeric columns types changed to the least possible size
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## 3. Create features on structures

In [6]:
!pip install tqdm
from tqdm import tqdm_notebook as tqdm
# get a reliable estimate of how long it will take.
# see immediately if it’s stuck.

# 3a. Bond counts by element
Based on the number of electrons in an atom's valence shell, we know how many bonds the atom needs to form to be stable. Hydrogen needs 1, Flourine needs 1, Oxygen needs 2, Nitrogen needs 3, and Carbon needs 4. Bonds can be single, double, or triple, but we have not yet calculated the strength of the bonds. Therefore there is a range of valid bond counts we could get for each atom.


# 3b. Using the atomic radii, electronegativity values of the atoms (C, H, N, O, F), 
new features like atoms_rad and atoms_en are created. The number of bonds as n_bonds and bond_lengths_mean. This gives a new structutres file.here we create a dictionary of a new column with the new values
then pass the values to the dataframe


# 3c. Fudge factor is to account for unceratainty
the fudge values are added slightly, in order to reduce false negatives. Atoms that are not bonded repel each other, so it should be rare that this increase will result in false positives.


In [7]:
%time
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor

fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}
print(atomic_radius)

electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

atoms = structures['atom'].values
atoms_en = [electronegativity[x] for x in tqdm(atoms)]
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

structures['EN'] = atoms_en
structures['rad'] = atoms_rad

i_atom = structures['atom_index'].values # a list of all 'atom_index' in structures
p = structures[['x', 'y', 'z']].values  # a list of cartesian coordinates (x,y,z) values in structures
p_compare = p
m = structures['molecule_name'].values # a list of molecule name in structures
m_compare = m
r = structures['rad'].values # a list of rad in structures
r_compare = r

source_row = np.arange(len(structures)) # returns an array of evenly spaced values
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule? We are trying to compare to 
                                          # to distinguish between two different molecules.
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
bond_lengths_mean = [ np.mean(x) for x in bond_lengths]
n_bonds = [len(x) for x in bonds_numeric]

bond_data = {'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean }
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs
{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


A Jupyter Widget

A Jupyter Widget


Calculating bonds


A Jupyter Widget


Counting and condensing bonds


A Jupyter Widget

A Jupyter Widget

,molecule_name,atom_index,atom,x,y,z,EN,rad,n_bonds,bond_lengths_mean
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,2.20,0.43,1,1.091953
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,2.20,0.43,1,1.091952
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,2.20,0.43,1,1.091946
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,2.20,0.43,1,1.091948
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,3.04,0.80,3,1.017195
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,2.20,0.43,1,1.017190
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,2.20,0.43,1,1.017187
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,2.20,0.43,1,1.017208
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,3.44,0.78,2,0.962107


# 4a. Function to merge structures data with train data 

In [8]:
def merge_train_structures(train, structures):
  
  """This function is used to merge the structures dataset to the 
     original train dataset
     
     Parameters:
     ===========
train: train dataframe
structures: structures dataframe.
     
     Output:  ===========
dataframe: merged dataframe
     """
  
  structures = structures.rename({'atom_index': 'atom_index_0',
                                  'x':'x_0', 'y':'y_0', 'z':'z_0',
                                  'atom':'atom_0'}, axis=1)
  
  merged = pd.merge(train, structures, on=['molecule_name', 'atom_index_0'])
  
  structures = structures.rename({'atom_index_0': 'atom_index_1',
                                  'x_0':'x_1', 'y_0':'y_1', 'z_0':'z_1',
                                  'atom_0':'atom_1'}, axis=1)
  
  merged_1 = pd.merge(merged, structures, on=['molecule_name', 'atom_index_1'])
  
  structures = structures.rename({'atom_index_1': 'atom_index',
                                  'x_1':'x', 'y_1':'y', 'z_1':'z',
                                  'atom_1':'atom'}, axis=1)
  
  assert train.shape[0] == merged.shape[0]
  
  return merged_1

In [9]:
merged = merge_train_structures(trainval, structures)

In [10]:
merged.head(1)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,n_bonds_x,bond_lengths_mean_x,atom_1,x_1,y_1,z_1,EN_y,rad_y,n_bonds_y,bond_lengths_mean_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,...,1,1.091953,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.09195


In [11]:
merged.shape

(4658147, 22)

In [12]:
assert len(merged) == len(trainval)
merged.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,n_bonds_x,bond_lengths_mean_x,atom_1,x_1,y_1,z_1,EN_y,rad_y,n_bonds_y,bond_lengths_mean_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,1,1.091953,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,1,1.091952,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,1,1.091946,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,1,1.091948,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,1,1.091953,H,1.011731,1.463751,0.000277,2.20,0.43,1,1.091952


# 4b. Function to merge Mulliken charges with previously merged dataset
Here the number of features have been increased to 24 from 22

In [13]:
mulliken = mulliken.rename({'atom_index': 'atom_index_0',
                          'mulliken_charge': 'mulliken_charge_0'}, axis=1)
merged = merged.merge(mulliken, on=['molecule_name', 'atom_index_0'])

mulliken = mulliken.rename({'atom_index_0': 'atom_index_1',
                          'mulliken_charge_0': 'mulliken_charge_1'}, axis=1)
merged = merged.merge(mulliken, on=['molecule_name', 'atom_index_1'])
assert len(merged) == len(trainval)

In [14]:
merged.head(1)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,atom_1,x_1,y_1,z_1,EN_y,rad_y,n_bonds_y,bond_lengths_mean_y,mulliken_charge_0,mulliken_charge_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,...,C,-0.012698,1.085804,0.008001,2.55,0.82,4,1.09195,0.133921,-0.535689


In [15]:
merged.shape

(4658147, 24)

## 4c. Creating features
1. distance — the distance between the given cartesian points of each atom
2. n_bonds — the number of bonds on a specific atom
3. mu — the square root of the sum of the squared Cartesian values
4. delta_en — the difference between the electronegativities of two atoms
5. Advanced features were created with the help of a few helpful Kaggle kernels

In [16]:
%%time
# This block is creating distance 'dist' feature using the linalg and merging with the 'merged' file.

train_p_0 = merged[['x_0', 'y_0', 'z_0']].values
train_p_1 = merged[['x_1', 'y_1', 'z_1']].values

merged['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)

assert len(merged) == len(trainval)#assert len means confirming no of items in merged and trainval

CPU times: user 540 ms, sys: 684 ms, total: 1.22 s
Wall time: 1.23 s


In [17]:
merged.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,x_1,y_1,z_1,EN_y,rad_y,n_bonds_y,bond_lengths_mean_y,mulliken_charge_0,mulliken_charge_1,dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950,0.133921,-0.535689,1.091953
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950,0.133922,-0.535689,1.091952
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950,0.133923,-0.535689,1.091946
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,-0.012698,1.085804,0.008001,2.55,0.82,4,1.091950,0.133923,-0.535689,1.091948
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,1.011731,1.463751,0.000277,2.20,0.43,1,1.091952,0.133921,0.133922,1.783120


# 4d. Grouping the data according to the molecule names

The training data includes more than 1 observation under each molecule_name. Because of this nature, we had to be considerate to not leak data from train molecules into the validation set. We did a train_test_split() on the molecule_name instead, and created train/val subsets with the data from each J type — now 16 subsets total!
Creating features like count, mean, min using distance

In [18]:
merged['type'].value_counts()
merged['molecule_couples'] = merged.groupby('molecule_name')['id'].transform('count')

In [19]:
 merged['molecule_dist_mean'] = merged.groupby('molecule_name')['dist'].transform('mean')

In [20]:
 merged['molecule_dist_min'] = merged.groupby('molecule_name')['dist'].transform('min')

In [21]:
external_data_0 = pd.DataFrame(data={
    'atom_0': ['C', 'H', 'N', 'O', 'F'],
    'atom_0_en': [2.55, 2.20, 3.04, 3.44, 3.98],
    'atomic_mass_0': [12.0107, 1.00784, 14.0067, 15.999, 18.9984],
    'valence_electrons_0': [4, 1, 5, 6, 7]
})
external_data_1 = pd.DataFrame(data={
    'atom_1': ['C', 'H', 'N', 'O', 'F'],
    'atom_1_en': [2.55, 2.20, 3.04, 3.44, 3.98],
    'atomic_mass_1': [12.0107, 1.00784, 14.0067, 15.999, 18.9984],
    'valence_electrons_1': [4, 1, 5, 6, 7]
})


    

In [22]:
#%%time
def create_features(df):
    df['bond'] = df['type'].str[2:]
    df['j_type'] = df['type'].str[:2]
    df['distance'] = ((df['x_1'].values - df['x_0'].values)**2 + 
                    (df['y_1'].values - df['y_0'].values)**2 + 
                    (df['z_1'].values - df['z_0'].values)**2)** 0.5
    df['mu_0'] = np.sqrt(df['x_0'].values**2 + df['y_0'].values**2 + df['z_0'].values**2)
    df['mu_1'] = np.sqrt(df['x_1'].values**2 + df['y_1'].values**2 + df['z_1'].values**2)
    df = df.merge(external_data_0, on='atom_0', how='left')
    df = df.merge(external_data_1, on='atom_1', how='left')
    df['delta_en'] = (df['atom_0_en'] - df['atom_1_en']).abs()
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['dist'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['dist'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['dist'].transform('max')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    df['molecule_atom_index_0_x_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_1'].transform('std')
    df['molecule_atom_index_0_y_1_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('mean')
    df['molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df['molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df['molecule_atom_index_0_y_1_max'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('max')
    df['molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df['molecule_atom_index_0_y_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('std')
    df['molecule_atom_index_0_z_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_1'].transform('std')
    df['molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('mean')
    df['molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df['molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['dist']
    df['molecule_atom_index_0_dist_max'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('max')
    df['molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['dist']
    df['molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['dist']
    df['molecule_atom_index_0_dist_min'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df['molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df['molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['dist']
    df['molecule_atom_index_0_dist_std'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('std')
    df['molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['dist']
    df['molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['dist']
    df['molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('mean')
    df['molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df['molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['dist']
    df['molecule_atom_index_1_dist_max'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('max')
    df['molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['dist']
    df['molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['dist']
    df['molecule_atom_index_1_dist_min'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('min')
    df['molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df['molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['dist']
    df['molecule_atom_index_1_dist_std'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('std')
    df['molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['dist']
    df['molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['dist']
    df['molecule_atom_1_dist_mean'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('mean')
    df['molecule_atom_1_dist_min'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('min')
    df['molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['dist']
    df['molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['dist']
    df['molecule_atom_1_dist_std'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('std')
    df['molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['dist']

    df = reduce_mem_usage(df)
    return df 


In [23]:
def map_atom_info(df_1,df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)

    return df

def create_closest(df_train):
    
    
    df_temp=df_train.loc[:,["molecule_name","atom_index_0","atom_index_1","dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_=df_temp.copy()
    df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
#     df_temp=pd.concat(objs=[df_temp,df_temp_],axis=0)

    df_temp["min_distance"]=df_temp.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df_temp= df_temp[df_temp["min_distance"]==df_temp["dist"]]

    df_temp=df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                     'atom_index_1': 'atom_index_closest',
                                     'distance': 'distance_closest',
                                     'x_1': 'x_closest',
                                     'y_1': 'y_closest',
                                     'z_1': 'z_closest'})

    for atom_idx in [0,1]:
        df_train = map_atom_info(df_train,df_temp, atom_idx)
        df_train = df_train.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
       
                                            
                                            
                                            'distance_closest': f'distance_closest_{atom_idx}',
                                            'x_closest': f'x_closest_{atom_idx}',
                                            'y_closest': f'y_closest_{atom_idx}',
                                            'z_closest': f'z_closest_{atom_idx}'})
    df_train = reduce_mem_usage(df_train)
    return df_train


In [24]:
#%%time Merging the 'merged' to the 'create_features' and asserting the length

merged = create_features(merged)
assert len(merged) == len(trainval)


Mem. usage decreased to 915.13 Mb (68.6% reduction)


In [25]:
#%%time
dtrain = create_closest(merged)

Mem. usage decreased to 999.53 Mb (2.6% reduction)


In [26]:
dtrain['dist'].isnull().sum(), dtrain['dist'].shape

(3689501, (4658147,))

In [27]:
dtrain['dist'].isnull().sum()

3689501

In [28]:
assert len(dtrain) == len(trainval)

In [29]:
dtrain.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,atom_index_closest_0,dist_y,x_closest_0,y_closest_0,z_closest_0,atom_index_closest_1,dist,x_closest_1,y_closest_1,z_closest_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.812500,H,0.002150,-0.006031,0.001976,...,0,1.091797,-0.012695,1.085938,0.008003,NaN,NaN,NaN,NaN,NaN
1,4,dsgdb9nsd_000001,2,0,1JHC,84.812500,H,1.011719,1.463867,0.000277,...,0,1.091797,-0.012695,1.085938,0.008003,NaN,NaN,NaN,NaN,NaN
2,7,dsgdb9nsd_000001,3,0,1JHC,84.812500,H,-0.541016,1.447266,-0.876465,...,0,1.091797,-0.012695,1.085938,0.008003,NaN,NaN,NaN,NaN,NaN
3,9,dsgdb9nsd_000001,4,0,1JHC,84.812500,H,-0.523926,1.437500,0.906250,...,0,1.091797,-0.012695,1.085938,0.008003,NaN,NaN,NaN,NaN,NaN
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.257812,H,0.002150,-0.006031,0.001976,...,0,1.091797,-0.012695,1.085938,0.008003,0.0,1.091797,-0.012695,1.085938,0.008003


# 4d. Create cosine, dihedral angle features 

In [30]:
def add_cos_features(df):
    df["distance_0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["distance_1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["vec_0_x"]=(df['x_0']-df['x_closest_0'])/df["distance_0"]
    df["vec_0_y"]=(df['y_0']-df['y_closest_0'])/df["distance_0"]
    df["vec_0_z"]=(df['z_0']-df['z_closest_0'])/df["distance_0"]
    df["vec_1_x"]=(df['x_1']-df['x_closest_1'])/df["distance_1"]
    df["vec_1_y"]=(df['y_1']-df['y_closest_1'])/df["distance_1"]
    df["vec_1_z"]=(df['z_1']-df['z_closest_1'])/df["distance_1"]
    df["vec_x"]=(df['x_1']-df['x_0'])/df["dist"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["dist"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["dist"]
    df["cos_0_1"]=df["vec_0_x"]*df["vec_1_x"]+df["vec_0_y"]*df["vec_1_y"]+df["vec_0_z"]*df["vec_1_z"]
    df["cos_0"]=df["vec_0_x"]*df["vec_x"]+df["vec_0_y"]*df["vec_y"]+df["vec_0_z"]*df["vec_z"]
    df["cos_1"]=df["vec_1_x"]*df["vec_x"]+df["vec_1_y"]*df["vec_y"]+df["vec_1_z"]*df["vec_z"]
    df=df.drop(['vec_0_x','vec_0_y','vec_0_z','vec_1_x','vec_1_y','vec_1_z','vec_x','vec_y','vec_z'], axis=1)
    df = reduce_mem_usage(df)
    return df
    
merged = add_cos_features(dtrain)


Mem. usage decreased to 1043.95 Mb (0.0% reduction)


In [31]:
assert len(merged) == len(trainval)
merged.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,atom_index_closest_1,dist,x_closest_1,y_closest_1,z_closest_1,distance_0,distance_1,cos_0_1,cos_0,cos_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.812500,H,0.002150,-0.006031,0.001976,...,NaN,NaN,NaN,NaN,NaN,1.091797,NaN,NaN,NaN,NaN
1,4,dsgdb9nsd_000001,2,0,1JHC,84.812500,H,1.011719,1.463867,0.000277,...,NaN,NaN,NaN,NaN,NaN,1.091797,NaN,NaN,NaN,NaN
2,7,dsgdb9nsd_000001,3,0,1JHC,84.812500,H,-0.541016,1.447266,-0.876465,...,NaN,NaN,NaN,NaN,NaN,1.091797,NaN,NaN,NaN,NaN
3,9,dsgdb9nsd_000001,4,0,1JHC,84.812500,H,-0.523926,1.437500,0.906250,...,NaN,NaN,NaN,NaN,NaN,1.091797,NaN,NaN,NaN,NaN
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.257812,H,0.002150,-0.006031,0.001976,...,0.0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333496,-1.333008,1.333984


In [32]:
# Now we havw a shape with 96 features but many 'dist' features have null values so we will delete them later 
#before modelling while selecting the features.

merged.shape


(4658147, 96)

## 5. Scoring Function 
we are using MAE as scoring function as it is robust against the outliers

In [33]:
groups = merged['type']

def group_lmae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true - y_pred).abs().groupby(groups).mean()
    return (maes.map(lambda x: max(x, floor))).mean()

## 6. Splitting the dataset according to coupling type
So we have 8 such coupling types and thus 8 such models

In [34]:
trainval_1JHC = merged[merged['type'] == '1JHC']
trainval_1JHN = merged[merged['type'] == '1JHN']
trainval_2JHH = merged[merged['type'] == '2JHH']
trainval_2JHN = merged[merged['type'] == '2JHN']
trainval_2JHC = merged[merged['type'] == '2JHC']
trainval_3JHH = merged[merged['type'] == '3JHH']
trainval_3JHC = merged[merged['type'] == '3JHC']
trainval_3JHN = merged[merged['type'] == '3JHN']

# 7. train_test_split of complete dataset (trainval_1JHC) for 1JHC type
This splitting is again based on molecules

X_train, X_test, y_train, y_test 
    = train_test_split(X, y, random_state=47)
    Here creating molecules array with unique ones passed for training and remaining passed for test.nso we have different molecules for training ang testing This is done for all the coupling types.

In [35]:
# all the molecules under the same coupling type 

train_1JHC, test_1JHC = train_test_split(trainval_1JHC, shuffle=False, random_state=47)

train_1JHC_molecules = train_1JHC['molecule_name'].unique()
test_1JHC_molecules = np.delete(test_1JHC['molecule_name'].unique(), 0)

train_1JHC = train_1JHC[train_1JHC['molecule_name'].isin(train_1JHC_molecules)]
test_1JHC = test_1JHC[test_1JHC['molecule_name'].isin(test_1JHC_molecules)]

## 8. Defining features and target
We can see above that there are several features with NaN values and we eliminate them before modeling

In [36]:
new_features_1 = ['EN_x', 'rad_x', 'n_bonds_x', 'bond_lengths_mean_x', 'EN_y', 'rad_y', 'n_bonds_y', 
                     'bond_lengths_mean_y','molecule_couples', 'molecule_dist_mean','molecule_dist_min', 'molecule_dist_max']
new_features_2 = ['atom_0_couples_count', 'atom_1_couples_count', 'molecule_atom_index_0_x_1_std', 
                  'molecule_atom_index_0_y_1_mean', 'molecule_atom_index_0_y_1_mean_diff', 
                  'molecule_atom_index_0_y_1_mean_div', 'molecule_atom_index_0_y_1_max',
                  'molecule_atom_index_0_y_1_max_diff', 'molecule_atom_index_0_y_1_std' ]
new_features_3 = ['molecule_atom_index_0_z_1_std',
                   'molecule_atom_index_0_dist_mean',
                   'molecule_atom_index_0_dist_mean_diff',
                   'molecule_atom_index_0_dist_mean_div',
                   'molecule_atom_index_0_dist_max',
                   'molecule_atom_index_0_dist_max_diff',
                   'molecule_atom_index_0_dist_max_div',
                   'molecule_atom_index_0_dist_min',
                   'molecule_atom_index_0_dist_min_diff',
                   'molecule_atom_index_0_dist_min_div',
                   'molecule_atom_index_0_dist_std',
                   'molecule_atom_index_0_dist_std_diff',
                   'molecule_atom_index_0_dist_std_div',
                   'molecule_atom_index_1_dist_mean',
                   'molecule_atom_index_1_dist_mean_diff',
                   'molecule_atom_index_1_dist_mean_div',
                   'molecule_atom_index_1_dist_max',
                   'molecule_atom_index_1_dist_max_diff',
                   'molecule_atom_index_1_dist_max_div',
                   'molecule_atom_index_1_dist_min',
                   'molecule_atom_index_1_dist_min_diff',
                   'molecule_atom_index_1_dist_min_div',
                   'molecule_atom_index_1_dist_std',
                   'molecule_atom_index_1_dist_std_diff',
                   'molecule_atom_index_1_dist_std_div',
                   'molecule_atom_1_dist_mean',
                   'molecule_atom_1_dist_min',
                   'molecule_atom_1_dist_min_diff',
                   'molecule_atom_1_dist_min_div',
                   'molecule_atom_1_dist_std',
                   'molecule_atom_1_dist_std_diff']

new_features_4 = ['atom_index_closest_0',
                   'dist_y',
                   'x_closest_0',
                   'y_closest_0',
                   'z_closest_0',
                   'atom_index_closest_1',
                   'dist',
                   'x_closest_1',
                   'y_closest_1',
                   'z_closest_1',
                   'distance_0',
                   'distance_1',
                   'cos_0_1',
                   'cos_0',
                   'cos_1']


features = ['atom_index_0', 'atom_index_1', 'distance', 'atom_0_en',
            'atom_1_en', 'mu_0', 'mu_1', 'mulliken_charge_0',
            'mulliken_charge_1', 'x_0', 'x_1', 'y_0', 'y_1', 'z_0', 'z_1'] + new_features_1 + new_features_2 + new_features_3 + new_features_4

drop_feat = [
    'molecule_atom_index_0_x_1_std',
    'molecule_atom_index_0_y_1_mean_div',
    'molecule_atom_index_0_y_1_std',
    'molecule_atom_index_0_z_1_std',
    'molecule_atom_index_0_dist_std',
    'molecule_atom_index_0_dist_std_diff',
    'molecule_atom_index_0_dist_std_div',
    'molecule_atom_index_1_dist_std',
    'molecule_atom_index_1_dist_std_diff',
    'molecule_atom_index_1_dist_std_div',
    'molecule_atom_1_dist_std',
    'molecule_atom_1_dist_std_diff',
    'atom_index_closest_1', 'dist', 'x_closest_1',
    'y_closest_1', 'z_closest_1', 'distance_1',
    'cos_0_1', 'cos_0', 'cos_1', 'EN_y', 'n_bonds_x',
    'molecule_atom_index_0_dist_min_diff',
    'molecule_atom_index_0_dist_min_div','rad_y', 'EN_x', 'rad_x',
    'dist_y', 'atom_1_en', 'atom_0_en', 'z_closest_0']

for i in drop_feat:
    features.remove(i)

    target = 'scalar_coupling_constant'



#full data
X_trainval_1JHC = trainval_1JHC[features]

y_trainval_1JHC = trainval_1JHC[target]


# split data
X_train_1JHC = train_1JHC[features]

y_train_1JHC = train_1JHC[target]

X_test_1JHC = test_1JHC[features]

y_test_1JHC = test_1JHC[target]


In [37]:
target

'scalar_coupling_constant'

In [38]:
X_trainval_1JHC.head()

,atom_index_0,atom_index_1,distance,mu_0,mu_1,mulliken_charge_0,mulliken_charge_1,x_0,x_1,y_0,...,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,atom_index_closest_0,x_closest_0,y_closest_0,distance_0
0,1,0,1.091797,0.006702,1.085938,0.133911,-0.535645,0.002150,-0.012695,-0.006031,...,-0.000007,1.0,1.091797,1.091797,-0.000007,1.0,0,-0.012695,1.085938,1.091797
1,2,0,1.091797,1.779297,1.085938,0.133911,-0.535645,1.011719,-0.012695,1.463867,...,-0.000005,1.0,1.091797,1.091797,-0.000005,1.0,0,-0.012695,1.085938,1.091797
2,3,0,1.091797,1.776367,1.085938,0.133911,-0.535645,-0.541016,-0.012695,1.447266,...,0.000000,1.0,1.091797,1.091797,0.000000,1.0,0,-0.012695,1.085938,1.091797
3,4,0,1.091797,1.778320,1.085938,0.133911,-0.535645,-0.523926,-0.012695,1.437500,...,-0.000001,1.0,1.091797,1.091797,-0.000001,1.0,0,-0.012695,1.085938,1.091797
17,2,0,1.066406,2.199219,1.132812,0.238159,-0.049652,-0.027802,-0.013321,2.199219,...,0.000000,1.0,1.066406,1.066406,0.000000,1.0,0,-0.013321,1.132812,1.066406


In [39]:
y_test_1JHC.head()

3537482    92.3750
3537483    92.1875
3537484    92.6250
3537500    92.5000
3537501    92.3125
Name: scalar_coupling_constant, dtype: float16

## 9. Encoding for the categorical values
After deleting features with NaN values we have now 50 features remaining

In [40]:
def encode(df):
    df = OrdinalEncoder.fit_transform(OrdinalEncoder(df), df)
    return df


In [41]:
#split data
X_train_1JHC_encoded = encode(X_train_1JHC)

X_test_1JHC_encoded = encode(X_test_1JHC)

#full data
X_trainval_1JHC_encoded = encode(X_trainval_1JHC)


In [42]:
X_train_1JHC_encoded.isnull().sum() 

atom_index_0                            0
atom_index_1                            0
distance                                0
mu_0                                    0
mu_1                                    0
mulliken_charge_0                       0
mulliken_charge_1                       0
x_0                                     0
x_1                                     0
y_0                                     0
y_1                                     0
z_0                                     0
z_1                                     0
bond_lengths_mean_x                     0
n_bonds_y                               0
bond_lengths_mean_y                     0
molecule_couples                        0
molecule_dist_mean                      0
molecule_dist_min                       0
molecule_dist_max                       0
atom_0_couples_count                    0
atom_1_couples_count                    0
molecule_atom_index_0_y_1_mean          0
molecule_atom_index_0_y_1_mean_dif

## 10. Applying the Model

Our hypothesis was that the coupling constant from each J type would be impacted differently by each feature, so we created 8 different models to get the most accurate predictions possible.

# 10 a. Linear regression

In [44]:
from sklearn.linear_model import LinearRegression

In [45]:
My_model=LinearRegression()

In [46]:
My_model.fit(X_train_1JHC,y_train_1JHC)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [47]:
y_pred = My_model.predict(X_test_1JHC)

In [48]:
y_pred

array([ 88.67627 ,  89.20703 ,  89.362305, ..., 109.20996 , 103.60791 ,
       110.86328 ], dtype=float32)

In [51]:
from sklearn.metrics import mean_absolute_error
print ('mean_absolute_error= ', mean_absolute_error(y_test_1JHC , y_pred))

mean_absolute_error=  4.58315


# 10 b. LightGBM (Microsoft)
Here the hyperparameters have been defined with a range of vlaues and the model is checking the results with these values.LightGBM here has been used with randomizedCV search.

In [43]:
# Defining thwe eavluation metrics as MAE (log)
def custom_eval_metric(y_true, y_pred):
    return 'custom_eval_metric', ((y_true - y_pred).abs().mean()), False

def custom_score_metric(y_true, y_pred, sample_weight):
    return ((y_true - y_pred).abs().mean())
  

# 1JHC LightGBM

In [44]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : custom_eval_metric, 
            "eval_set" : [(X_test_1JHC, y_test_1JHC)],
            'eval_names': ['valid'],
            'verbose': 1,
            'categorical_feature': 'auto'}

param_test ={'num_leaves': [x for x in range(0, 100, 10)], 
             'min_child_samples': [x for x in range(100, 500, 10)], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}


clf_1JHC = lgb.LGBMRegressor(max_depth=-1, random_state=47, n_jobs=2, n_estimators=50)
                          

gs_1JHC = RandomizedSearchCV(
    estimator=clf_1JHC,
    param_distributions=param_test, 
    n_iter=3,
    cv=3,
    refit=True,
    random_state=47)

gs_1JHC.fit(X_train_1JHC, y_train_1JHC, **fit_params)

final_params_1JHC = gs_1JHC.best_params_
final_params_1JHC['n_estimators'] = 2000
final_params_1JHC['max_depth'] = -1
final_params_1JHC['random_state'] = 47
final_params_1JHC['n_jobs'] = -1 # this could be reduced to 2-4 range as -1 is usimg all the cores to perform parallel search.


clf_1JHC = lgb.LGBMRegressor()

clf_1JHC.set_params(**final_params_1JHC)

clf_1JHC.fit(X_train_1JHC, y_train_1JHC)

y_pred_1JHC = clf_1JHC.predict(X_test_1JHC)

pred_vs_actual_1JHC = pd.DataFrame(data={
    'predictions': y_pred_1JHC,
    'actual': y_test_1JHC
})

[1]	valid's l2: 230.319	valid's custom_eval_metric: 10.5709
Training until validation scores don't improve for 30 rounds.
[2]	valid's l2: 198.175	valid's custom_eval_metric: 9.72646
[3]	valid's l2: 171.62	valid's custom_eval_metric: 8.96326
[4]	valid's l2: 146.382	valid's custom_eval_metric: 8.22746
[5]	valid's l2: 127.461	valid's custom_eval_metric: 7.62758
[6]	valid's l2: 112.799	valid's custom_eval_metric: 7.12244
[7]	valid's l2: 96.8589	valid's custom_eval_metric: 6.5695
[8]	valid's l2: 85.5671	valid's custom_eval_metric: 6.13093
[9]	valid's l2: 76.6591	valid's custom_eval_metric: 5.75774
[10]	valid's l2: 69.8408	valid's custom_eval_metric: 5.4508
[11]	valid's l2: 64.5573	valid's custom_eval_metric: 5.24191
[12]	valid's l2: 58.2203	valid's custom_eval_metric: 4.97567
[13]	valid's l2: 54.3196	valid's custom_eval_metric: 4.79428
[14]	valid's l2: 50.8486	valid's custom_eval_metric: 4.62814
[15]	valid's l2: 47.3385	valid's custom_eval_metric: 4.47316
[16]	valid's l2: 45.0586	valid's cu

[30]	valid's l2: 23.1893	valid's custom_eval_metric: 3.25395
[31]	valid's l2: 22.8132	valid's custom_eval_metric: 3.224
[32]	valid's l2: 22.5003	valid's custom_eval_metric: 3.20313
[33]	valid's l2: 22.0953	valid's custom_eval_metric: 3.17413
[34]	valid's l2: 21.7818	valid's custom_eval_metric: 3.15048
[35]	valid's l2: 21.6535	valid's custom_eval_metric: 3.14263
[36]	valid's l2: 21.5811	valid's custom_eval_metric: 3.13968
[37]	valid's l2: 21.5595	valid's custom_eval_metric: 3.14089
[38]	valid's l2: 21.6384	valid's custom_eval_metric: 3.14339
[39]	valid's l2: 21.3606	valid's custom_eval_metric: 3.12219
[40]	valid's l2: 21.1021	valid's custom_eval_metric: 3.10441
[41]	valid's l2: 20.837	valid's custom_eval_metric: 3.08568
[42]	valid's l2: 20.5226	valid's custom_eval_metric: 3.06682
[43]	valid's l2: 20.3433	valid's custom_eval_metric: 3.05367
[44]	valid's l2: 20.3605	valid's custom_eval_metric: 3.05471
[45]	valid's l2: 20.1336	valid's custom_eval_metric: 3.03947
[46]	valid's l2: 19.7923	va

[7]	valid's l2: 79.5847	valid's custom_eval_metric: 6.82123
[8]	valid's l2: 67.3684	valid's custom_eval_metric: 6.27087
[9]	valid's l2: 57.3222	valid's custom_eval_metric: 5.77273
[10]	valid's l2: 49.2583	valid's custom_eval_metric: 5.34643
[11]	valid's l2: 42.6521	valid's custom_eval_metric: 4.96372
[12]	valid's l2: 37.1328	valid's custom_eval_metric: 4.62431
[13]	valid's l2: 32.6042	valid's custom_eval_metric: 4.31599
[14]	valid's l2: 28.8059	valid's custom_eval_metric: 4.04036
[15]	valid's l2: 25.7237	valid's custom_eval_metric: 3.79989
[16]	valid's l2: 23.2648	valid's custom_eval_metric: 3.59345
[17]	valid's l2: 21.1582	valid's custom_eval_metric: 3.41285
[18]	valid's l2: 19.4832	valid's custom_eval_metric: 3.2593
[19]	valid's l2: 17.9852	valid's custom_eval_metric: 3.11288
[20]	valid's l2: 16.8051	valid's custom_eval_metric: 2.99373
[21]	valid's l2: 15.807	valid's custom_eval_metric: 2.89037
[22]	valid's l2: 14.9154	valid's custom_eval_metric: 2.79389
[23]	valid's l2: 14.1956	vali

[37]	valid's l2: 9.24594	valid's custom_eval_metric: 2.10634
[38]	valid's l2: 9.13459	valid's custom_eval_metric: 2.09195
[39]	valid's l2: 9.0522	valid's custom_eval_metric: 2.0806
[40]	valid's l2: 8.96975	valid's custom_eval_metric: 2.07053
[41]	valid's l2: 8.89384	valid's custom_eval_metric: 2.05939
[42]	valid's l2: 8.82174	valid's custom_eval_metric: 2.05048
[43]	valid's l2: 8.73096	valid's custom_eval_metric: 2.03999
[44]	valid's l2: 8.67273	valid's custom_eval_metric: 2.03349
[45]	valid's l2: 8.59759	valid's custom_eval_metric: 2.02366
[46]	valid's l2: 8.5333	valid's custom_eval_metric: 2.01523
[47]	valid's l2: 8.49153	valid's custom_eval_metric: 2.0104
[48]	valid's l2: 8.41402	valid's custom_eval_metric: 1.99986
[49]	valid's l2: 8.36252	valid's custom_eval_metric: 1.993
[50]	valid's l2: 8.29855	valid's custom_eval_metric: 1.98416
Did not meet early stopping. Best iteration is:
[50]	valid's l2: 8.29855	valid's custom_eval_metric: 1.98416
[1]	valid's l2: 236.118	valid's custom_eval

# 10.c Random forest model

In [45]:
model = RandomForestRegressor()
model.fit(X_train_1JHC,y_train_1JHC)

/home/saritacharde04/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [47]:
y_pred_1JHC = model.predict(X_test_1JHC)
MAE = mean_absolute_error(y_test_1JHC , y_pred_1JHC)
print('Random forest validation MAE = ', MAE)

Random forest validation MAE =  1.354278426519013


# 10.d XGBoost model

In [60]:
!pip install xgboost

     |████████████████████████████████| 142.8MB 3.9MB/s eta 0:00:01


In [61]:
#%% time
import xgboost as xgb
from xgboost import XGBRegressor
XGBModel = XGBRegressor()
XGBModel.fit(X_train_1JHC,y_train_1JHC , verbose=False)


# Get the mean absolute error on the validation data

# In[ ]:


y_pred_1JHC = XGBModel.predict(X_test_1JHC)
MAE = mean_absolute_error(y_test_1JHC , y_pred_1JHC)
print('XGBoost validation MAE = ',MAE)


/home/saritacharde04/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/saritacharde04/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[05:39:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost validation MAE =  2.8172913


# 10e. Catboost

In [62]:
!pip install catboost

     |████████████████████████████████| 61.9MB 777kB/s eta 0:00:01     |█████████████                   | 25.4MB 1.7MB/s eta 0:00:22
     |████████████████████████████████| 7.1MB 40.8MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp36-none-any.whl size=9533 sha256=e4792d23c501c9a046bead053ba8167ab49133a596be38e2b3c64dd33fe609c7
  Stored in directory: /home/saritacharde04/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [63]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [30, 50, 100]
                 }
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train_1JHC,y_train_1JHC)    

MAE = mean_absolute_error(y_test_1JHC , y_pred_1JHC)
print('CatBoost validation MAE = ',MAE)

/home/saritacharde04/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0:	learn: 87.7512429	total: 95.2ms	remaining: 9.43s
1:	learn: 79.0787462	total: 180ms	remaining: 8.84s
2:	learn: 71.2512428	total: 250ms	remaining: 8.08s
3:	learn: 64.1972536	total: 309ms	remaining: 7.42s
4:	learn: 57.8606957	total: 369ms	remaining: 7.01s
5:	learn: 52.1734357	total: 437ms	remaining: 6.85s
6:	learn: 47.0428253	total: 510ms	remaining: 6.78s
7:	learn: 42.4291699	total: 581ms	remaining: 6.68s
8:	learn: 38.2809857	total: 658ms	remaining: 6.65s
9:	learn: 34.5622582	total: 722ms	remaining: 6.5s
10:	learn: 31.2205594	total: 798ms	remaining: 6.46s
11:	learn: 28.2119583	total: 870ms	remaining: 6.38s
12:	learn: 25.5090792	total: 944ms	remaining: 6.32s
13:	learn: 23.0822513	total: 1.01s	remaining: 6.23s
14:	learn: 20.9271964	total: 1.06s	remaining: 6.01s
15:	learn: 18.9875176	total: 1.13s	remaining: 5.94s
16:	learn: 17.2696464	total: 1.2s	remaining: 5.84s
17:	learn: 15.7248344	total: 1.27s	remaining: 5.79s
18:	learn: 14.3411068	total: 1.34s	remaining: 5.73s
19:	learn: 13.1081605	t

# 11. Inference: The mean absolute error (MAE) and time taken, for different models are listed below. 


Linear regression:4.58315
LightGBM:1.99(2min 28sec)
Random Forest : 1.35(4min 04 sec)
XGBoost: 2.81(1min 9sec)
catboost: 2.81(5 min)

Modeling
The hypothesis was that the coupling constant from each J type, lie in a definite range of values, and would be impacted differently by each feature, so we created 8 different models to get the most accurate predictions possible. Here only one model is being comparing.
I first started with a simple Linear Regression for a baseline model, but needed something with decision trees for better accuracy. It is also confirmed that using 8 models was significantly better than 1 by running a test model with all of the J types in the data for comparison.
Ultimately, a LightGBM Regressor model was used for all 8 J types. It was then tuned for the best hyper parameters, by running the LGBM Regressor with early_stopping_rounds and RandomizedSearchCV to get the best score possible.

I have compared the results obtained by applying various models. Linear regresssion is not useful but the other boosting models are not much different and time taken by XGBoost is the least.
This could be different for different coupling types.